# CS486 - Artificial Intelligence
## Lesson 12 - Expectimax

Minimax is used to reason about the outcomes of adversarial decisions. We can use the same strategy to reason about the outcomes of chance.  

In [1]:
import helpers
from aima.games import *
from aima.notebook import psource

## Draw HiLo

Let's use expectimax to play Draw HiLo, is card game where you guess if successive cards will be higher or lower than the previes card. First, let's walk through an AIMA implementation of the game:

In [494]:
from random import randint

class HiLo(StochasticGame):
    def __init__(self):
        self.initial = StochasticGameState(
            to_move=1,
            utility=1,
            board=[randint(2,12)],
            moves=[-1,1], 
            chance=None
        )
    
    def chances(self,state):
        return list(range(1,14))

    def probability(self,chance):
        return 1/13
    
    def outcome(self,state,chance):
        return StochasticGameState(
            to_move=1,
            utility=state.utility,
            board=state.board.copy(),
            moves=[-1,1], 
            chance=chance
        )

    def result(self,state,action):
        card = state.board[-1]
        draw = state.chance
        hilo = (draw>card)-(draw<card)
        #print("card",card,"draw",draw,"hilo",hilo,"action",action)
        if hilo == 0:
            utility = state.utility
        elif hilo == action:
            utility = state.utility*2
        else:
            utility = -1
        
        return StochasticGameState(
            to_move=1,
            utility=utility,
            board=state.board + [draw],
            moves=[-1,1], 
            chance=None
        )
    
    def utility(self,state,player):
        return state.utility
    
    def actions(self,state):
        return state.moves
    
    def terminal_test(self,state):
        return len(state.board) == 5 or state.utility < 0
    
    def display(self,state):
        card = state.board[-1]
        print("Card:", card, "Pot:", state.utility)

So we've defined what the odds of 

In [411]:
psource(expectiminimax)

In [512]:
def expectmax_player(game, state):
    def max_value(state):
        v = -infinity
        for a in game.actions(state):
            v = max(v, chance_node(state, a))
        return v

    def chance_node(state, action):
        sum_chances = 0
        num_chances = len(game.chances(state))

        for chance in game.chances(state):
            res_state = game.outcome(state, chance)
            res_state = game.result(res_state, action)
            
            if game.terminal_test(res_state):
                util = game.utility(res_state, player)
            else:
                util = max_value(res_state)
                
            sum_chances += util * game.probability(chance)
                        
        return sum_chances / num_chances

    game.display(state)
    
    return argmax(game.actions(state),
                  key=lambda a: chance_node(state, a), default=None)

In [464]:
hilo = HiLo()
hilo.play_game(query_player)

current state:
Card: 9 Pot: 1
available moves: [-1, 1]

Card: 1 Pot: 2


2

In [ ]:
minimax_player = lambda game,state: minimax_decision(state,game)
mininim.play_game(query_player,minimax_player)

In [ ]:
minimax_player = lambda game,state: minimax_decision(state,game)
mininim.play_game(minimax_player,query_player)

In [516]:
def player(game,state):
    game.display(state)
    return expectiminimax(state,game)
    return move

HiLo().play_game(expectmax_player)

Card: 4 Pot: 1
Card: 5 Pot: 2
Card: 5 Pot: 2
Card: 11 Pot: 4
Card: 10 Pot: 8


8